## Résultats de votation dans les communes vaudoises

https://www.elections.vd.ch/votelec/app21/index.html?id=CHVO20241124#v=results&m=downLoadFiles

Mode d’emploi:
1. Créer un "API Token" dans Datawrapper: https://app.datawrapper.de/account/api-tokens et copier sa valeur. Attention, il ne faut pas publier cette clé (sur GitHub ou ailleurs sur le Web)
2. Dans Datawrapper encore, créer un graphique de base, qu'on utilisera comme modèle; mettre son ID Datawrapper de côté. Le modèle que j’ai utilisé est: https://datawrapper.dwcdn.net/8tMFF/1/
3. Tester la génération en série de 2-3 graphiques
4. Si ça fonctionne, vous pouvez la lancer pour tous vos graphiques

In [3]:
import pandas as pd
import requests

In [ ]:
KEY = "VOTRE-CLE-D-API" # Ne pas “pusher” dans le repo si vous mettez votre clé d’API ici
HEADERS = {"accept": "*/*", "Authorization": f"Bearer {KEY}"}

In [51]:
df = pd.read_excel("data/CHVO20241124-01.xlsx", skiprows=1)

In [57]:
# Elimination de lignes vides ou inutiles
df.dropna(subset="Communes", inplace=True)

In [ ]:
# On ajoute une colonne district. Je n'ai PAS VERIFIE que le resultat soit entièrement correct, donc je vous
# laisse ce soin si vous utilisez ce script (par exemple en exportant en Excel pour regarder de près)

df["District"] = df["Communes"].apply(lambda x: x if "District" in x else None)
df["District"].ffill(inplace=True)

In [67]:
# On ne conserve que les lignes avec les communes ou Suisses de l’étranger
df = df[df["Bulletins rentrés"].notna()].copy()
df

,Communes,Electeurs inscrits,Bulletins rentrés,Blancs,Nuls,Valables,OUI,%,NON,%.1,Part. \nen %,District
2,Aigle,5379.0,2181.0,42.0,0.0,2139.0,811.0,NaN,1328.0,NaN,NaN,District Aigle
3,Bex,4571.0,1864.0,19.0,0.0,1845.0,667.0,NaN,1178.0,NaN,NaN,District Aigle
4,Chessel,310.0,155.0,2.0,0.0,153.0,76.0,NaN,77.0,NaN,NaN,District Aigle
5,Corbeyrier,340.0,173.0,3.0,1.0,169.0,57.0,NaN,112.0,NaN,NaN,District Aigle
6,Gryon,877.0,415.0,3.0,0.0,412.0,184.0,NaN,228.0,NaN,NaN,District Aigle
...,...,...,...,...,...,...,...,...,...,...,...,...
325,Rossinière,343.0,144.0,1.0,0.0,143.0,57.0,NaN,86.0,NaN,NaN,District Riviera - Pays-d'Enhaut
326,Rougemont,449.0,169.0,0.0,0.0,169.0,87.0,NaN,82.0,NaN,NaN,District Riviera - Pays-d'Enhaut
327,Vevey,9915.0,4901.0,95.0,11.0,4795.0,1403.0,NaN,3392.0,NaN,NaN,District Riviera - Pays-d'Enhaut
328,Veytaux,529.0,223.0,4.0,0.0,219.0,101.0,NaN,118.0,NaN,NaN,District Riviera - Pays-d'Enhaut


In [68]:
# Conversion en entier
df["OUI"] = df["OUI"].astype(int)
df["NON"] = df["NON"].astype(int)

## Deux fonctions pour interagir avec Datawrapper

* `duplicate_chart()` dupliquera notre modèle

* `update_chart()` nous permettra de changer les données et la titraille

In [69]:
def duplicate_chart(chart_id):
    """Dupliquer un graphique et retourner l’ID du nouveau graphique"""
    response_duplicate = requests.post(
        f"https://api.datawrapper.de/v3/charts/{chart_id}/copy", headers=HEADERS
    )
    response_json = response_duplicate.json()
    new_chart_id = response_json.get("publicId")
    return new_chart_id

In [86]:
def update_chart(chart_id, data_encoded, title, lead):
    """Ajouter des donnees a un graphique"""
    base_url = f"https://api.datawrapper.de/v3/charts/{chart_id}"

    # 1) On ajoute les donnees
    response_data = requests.put(
        f"{base_url}/data",
        data=data_encoded,
        headers=HEADERS,
    )
    if response_data.status_code // 10 != 20:
        print(f"Erreur lors de l'envoi des donnees: {response_data.status_code}")
        return False

    # 2) on change le titre et le lead
    response_metadata = requests.patch(
        base_url,
        json={
            "title": title,
            "metadata": {
                "describe": {"intro": lead},
            },
        },
        headers=HEADERS,
    )
    if response_metadata.status_code // 10 != 20:
        print(f"Erreur lors du changment de titraille: {response_data.status_code}")
        return False

In [79]:
# Export des données pour un seul graphique: on les mets
# dans le presse-papier pour les copier-coller dans DataWrapper
# et créer notre modèle
for district_name, group in df.groupby("District"):
    group[["Communes", "OUI", "NON"]].to_clipboard(index=False)
    break

In [85]:
group.to_csv(index=False).encode("utf-8")

b'Communes,Electeurs inscrits,Bulletins rentr\xc3\xa9s,Blancs,Nuls,Valables,OUI,%,NON,%.1,"Part. \nen %",District\nAigle,5379.0,2181.0,42.0,0.0,2139.0,811,,1328,,,District Aigle\nBex,4571.0,1864.0,19.0,0.0,1845.0,667,,1178,,,District Aigle\nChessel,310.0,155.0,2.0,0.0,153.0,76,,77,,,District Aigle\nCorbeyrier,340.0,173.0,3.0,1.0,169.0,57,,112,,,District Aigle\nGryon,877.0,415.0,3.0,0.0,412.0,184,,228,,,District Aigle\nLavey-Morcles,627.0,274.0,3.0,0.0,271.0,106,,165,,,District Aigle\nLeysin,1480.0,658.0,13.0,0.0,645.0,247,,398,,,District Aigle\nNoville,709.0,304.0,5.0,1.0,298.0,154,,144,,,District Aigle\nOllon VD,4344.0,2243.0,34.0,0.0,2209.0,882,,1327,,,District Aigle\nOrmont-Dessous,811.0,355.0,2.0,0.0,353.0,134,,219,,,District Aigle\nOrmont-Dessus,919.0,460.0,13.0,0.0,447.0,202,,245,,,District Aigle\nRennaz,495.0,188.0,1.0,0.0,187.0,74,,113,,,District Aigle\nRoche VD,984.0,348.0,5.0,7.0,336.0,135,,201,,,District Aigle\nVilleneuve VD,2798.0,1146.0,22.0,0.0,1124.0,441,,683,,,District 

In [88]:
# Ensuite, on peut tester avec quelques autres districts
counter = 0
limit = 3
for district_name, group in df.groupby("District"):
    new_chart_id = duplicate_chart("8tMFF")
    df_encoded = group[["Communes", "OUI", "NON"]].to_csv(index=False).encode("utf-8")
    update_chart(new_chart_id, df_encoded, f"Résultats dans le {district_name}", "Ici un petit chapeau")
    print(f"Graphique créé pour le {district_name}! ID: {new_chart_id}, éditer le graphique: https://app.datawrapper.de/edit/{new_chart_id}/visualize#refine")
    counter += 1
    if counter > limit:
        break

Graphique créé pour le District Aigle! ID: aS51B, éditer le graphique: https://app.datawrapper.de/edit/aS51B/visualize#refine
Graphique créé pour le District Broye - Vully! ID: 7yEgb, éditer le graphique: https://app.datawrapper.de/edit/7yEgb/visualize#refine
Graphique créé pour le District Gros-de-Vaud! ID: q3Bnc, éditer le graphique: https://app.datawrapper.de/edit/q3Bnc/visualize#refine
Graphique créé pour le District Jura - Nord vaudois! ID: xuP8F, éditer le graphique: https://app.datawrapper.de/edit/xuP8F/visualize#refine


Et voilà! 

Il faudrait rendre les noms des districs plus jolis. Vous pouvez modifier les noms dans le fichier Excel d'origine ou ajouter une ligne en Python.

Et quand tout est au point, on peut lancer le script pour tous les districts. Le même principe peut s’appliquer à plein d'autres données.